In [ ]:
pip install -r requirements.txt
best_model = pd.read_pickle("data/best_model.pkl")

In [ ]:
import json
import pickle
import numpy as np

# Load the best model once when the Lambda container initializes.
# Make sure best_model.pkl is packaged with your deployment bundle.
with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

def lambda_handler(event, context):
    """
    AWS Lambda handler function.
    
    Expects an HTTP POST request with a JSON body containing a 'features' key,
    where the value is a list of feature values for prediction.
    """
    try:
        # Parse input
        body = json.loads(event.get('body', '{}'))
        features = body.get('features', None)
        if features is None:
            raise ValueError("Missing 'features' in request body.")
        
        # Convert features to numpy array and reshape as needed.
        input_data = np.array(features).reshape(1, -1)
        
        # Make prediction using the best_model.
        prediction = best_model.predict(input_data)
        
        # Prepare response.
        response = {
            'statusCode': 200,
            'body': json.dumps({
                'prediction': prediction.tolist()  # Ensure numpy types are converted to Python types
            })
        }
    except Exception as e:
        # If there is an error, return an error message.
        response = {
            'statusCode': 400,
            'body': json.dumps({'error': str(e)})
        }
    
    return response
